## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

In [10]:
import mne

# tarefa 1
data = mne.read_epochs("../../datasets/beta/beta_epo.fif")

Reading c:\Users\Maria Fernanda\Desktop\SSVEP\src\beta\..\..\datasets\beta\beta_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


In [11]:
# TAREFA 2
threshold = 0.25

# entendimento dos dados
print(data.get_data().shape)
data

(160, 64, 750)


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 2.996 s
Baseline,off


In [41]:
import numpy as np

targets = [float(item) for item in data.event_id.keys()]
print("Possíveis alvos:", targets)

features = list()
for i in range(len(data)):
    sample = list()
    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        test = data[i].compute_psd(method='welch', fmin=fmin,
                                          fmax=fmax, verbose=False)
        sample.append(test)
    features.append(sample)

Possíveis alvos: [8.0, 8.2, 8.4, 8.6, 8.799999999999999, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.600000000000001, 12.8, 13.0, 13.200000000000001, 13.4, 13.600000000000001, 13.8, 14.0, 14.200000000000001, 14.4, 14.600000000000001, 14.8, 15.0, 15.200000000000001, 15.4, 15.600000000000001, 15.8]


In [40]:
features
# X = np.array([features])
# print("Formato dos dados calculados:", X)

[[array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtype=float64),
  array([], shape=(1, 64, 0), dtyp

In [42]:
# organizando os dados

X = X.reshape(X.shape[0], X.shape[1], X.shape[-1])
print("Formato padronizado dos dados com PSD calculado:", X.shape)

Formato padronizado dos dados com PSD calculado: (160, 40, 0)


In [43]:
# TAREFA 4

# aplicando a característica de "maior valor"

max_values = np.max(X, axis=-1)
max_values.shape

ValueError: zero-size array to reduction operation maximum which has no identity

In [45]:
# TAREFA 5

y = np.load("../../datasets/beta/labels.npy")

# entendimento dos dados
print(y, y.shape, targets)

# classificação pelo "voto" do maior valor
i_max = max_values.argmax(axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print(f'\nPorcentagem de acerto: {acc:.2f}%')

[ 8.6  8.6  8.6  8.6  8.8  8.8  8.8  8.8  9.   9.   9.   9.   9.2  9.2
  9.2  9.2  9.4  9.4  9.4  9.4  9.6  9.6  9.6  9.6  9.8  9.8  9.8  9.8
 10.  10.  10.  10.  10.2 10.2 10.2 10.2 10.4 10.4 10.4 10.4 10.6 10.6
 10.6 10.6 10.8 10.8 10.8 10.8 11.  11.  11.  11.  11.2 11.2 11.2 11.2
 11.4 11.4 11.4 11.4 11.6 11.6 11.6 11.6 11.8 11.8 11.8 11.8 12.  12.
 12.  12.  12.2 12.2 12.2 12.2 12.4 12.4 12.4 12.4 12.6 12.6 12.6 12.6
 12.8 12.8 12.8 12.8 13.  13.  13.  13.  13.2 13.2 13.2 13.2 13.4 13.4
 13.4 13.4 13.6 13.6 13.6 13.6 13.8 13.8 13.8 13.8 14.  14.  14.  14.
 14.2 14.2 14.2 14.2 14.4 14.4 14.4 14.4 14.6 14.6 14.6 14.6 14.8 14.8
 14.8 14.8 15.  15.  15.  15.  15.2 15.2 15.2 15.2 15.4 15.4 15.4 15.4
 15.6 15.6 15.6 15.6 15.8 15.8 15.8 15.8  8.   8.   8.   8.   8.2  8.2
  8.2  8.2  8.4  8.4  8.4  8.4] (160,) [8.0, 8.2, 8.4, 8.6, 8.799999999999999, 9.0, 9.2, 9.4, 9.6, 9.8, 10.0, 10.2, 10.4, 10.6, 10.8, 11.0, 11.2, 11.4, 11.6, 11.8, 12.0, 12.2, 12.4, 12.600000000000001, 12.8, 13.0, 13.2000

TypeError: object of type 'numpy.int64' has no len()